In [ ]:
!pip install langchain==0.1.20
!pip install langchain_community==0.0.38
!pip install langchain_core==0.1.52
!pip install langchain_groq==0.1.3
!pip install langchain_openai==0.1.6
!pip install langchain_text_splitters==0.0.1


In [ ]:
pip install psycopg2

In [ ]:
!pip install sentence-transformers

In [ ]:
pip install faiss-cpu

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever
# import streamlit as st
import os
from langchain_groq import ChatGroq
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
!pip install PyPDF2


In [ ]:
from PyPDF2 import PdfReader
import os

In [ ]:
os.environ["GROQ_API_KEY"] = "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"

In [ ]:
llm = ChatGroq(temperature=1, model_name="llama3-70b-8192")

In [ ]:
uploaded_file = '/content/سوالات.pdf'

In [ ]:
def get_pdf_text(pdf):
    pdf_reader = PdfReader(pdf)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()
    return text

In [ ]:
data  = get_pdf_text('/content/شخص دوستان.pdf')

In [ ]:
# type(filem)

In [ ]:
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=900,
        chunk_overlap=100,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
# splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=100)
splits = get_text_chunks(data)

In [ ]:
splits

['محمدرضا طالع اکمل معروف به ایگو، طول اکمل، عرض اکمل و مارمولک تخمی متولد 1377   اهل\nکرج است. او به شدت کونی است و به همه کون می دهد. کافی است به او پول بدهی؛ در این صورت  \nاو کیر را در حلقش فرو کرده و به صورت مجلسی ساک میزند. در حال حاضر در شرکت اسنپ کار   \nمیکند. در ان  جا می گوزد و عن می خورد. او قبال در شرکت یکتانت مشغول بود و در آن جا به مدیرش   \nکون میداد.  حقوق او در شرکت اسنپ   20 .میلیون است \nشایان محمدی معروف به کواال، عمه جنده، فیس تخمی و چوب عنی است. او با معدل کیری 14   به زور\nفارغ التحصیل شد. او در حال حاضر در شرکت خدمات انفورماتیک به عنوان کارشناس تست، هیچ  \nکاری انجام نمی دهد و فقط به کاشیها نگاه می کند و پول مفت می گیرد. او مثل سگ کار میکند و  60  \n ساعت اضافه کار پر میکند. شایان محمدی به هیچ دردی نمیخورد و فقط با صورت تخمی اش شهر را   \nزشت کرده است. برجسته ترین ویژگی های او عبارت است: تخمی بودن، عن بودن، کیری بودن و عمه',
 'زشت کرده است. برجسته ترین ویژگی های او عبارت است: تخمی بودن، عن بودن، کیری بودن و عمه   \nجنده بودن   و سیگاری بودن  است.  حقوقش او حدود

In [ ]:
def get_vectorstore(text_chunks):
    embeddings = HuggingFaceEmbeddings()
    knowledge_base = FAISS.from_texts(text_chunks,embeddings)
    return knowledge_base

In [ ]:
vector_db = get_vectorstore(splits)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when p

In [ ]:
 QUERY_PROMPT = PromptTemplate(
        input_variables=["question"],
        template="""You are an Iranian AI language model assistant. Your task is to generate three
            different versions of the given user question to retrieve relevant documents from
            a vector database. By generating multiple perspectives on the user question, your
            goal is to help the user overcome some of the limitations of the distance-based
            similarity search. Provide these alternative questions separated by newlines.
            Original question: {question}""",
    )

In [ ]:
retriever = MultiQueryRetriever.from_llm(
        vector_db.as_retriever(),
        llm,
        prompt=QUERY_PROMPT
    )

In [ ]:
data

'  محمدرضا طالع اکمل معروف به ایگو، طول اکمل، عرض اکمل و مارمولک تخمی متولد 1377   اهل\nکرج است. او به شدت کونی است و به همه کون می دهد. کافی است به او پول بدهی؛ در این صورت  \nاو کیر را در حلقش فرو کرده و به صورت مجلسی ساک میزند. در حال حاضر در شرکت اسنپ کار   \nمیکند. در ان  جا می گوزد و عن می خورد. او قبال در شرکت یکتانت مشغول بود و در آن جا به مدیرش   \nکون میداد.  حقوق او در شرکت اسنپ   20 .میلیون است \nشایان محمدی معروف به کواال، عمه جنده، فیس تخمی و چوب عنی است. او با معدل کیری 14   به زور\nفارغ التحصیل شد. او در حال حاضر در شرکت خدمات انفورماتیک به عنوان کارشناس تست، هیچ  \nکاری انجام نمی دهد و فقط به کاشیها نگاه می کند و پول مفت می گیرد. او مثل سگ کار میکند و  60  \n ساعت اضافه کار پر میکند. شایان محمدی به هیچ دردی نمیخورد و فقط با صورت تخمی اش شهر را   \nزشت کرده است. برجسته ترین ویژگی های او عبارت است: تخمی بودن، عن بودن، کیری بودن و عمه   \nجنده بودن   و سیگاری بودن  است.  حقوقش او حدود 50  میلیون است و از بقیه دوستانش بیشتر حقوق می-\nگیرد.  \nآرمین بدرخانی معروف به بوفالو،

In [ ]:
WRITER_SYSTEM_PROMPT = "You are an IRANIAN AI critical thinker research assistant. Your sole purpose is to write well written, critically acclaimed, objective and structured reports on given text."  # noqa: E501
    # Report prompts from https://github.com/assafelovic/gpt-researcher/blob/master/gpt_researcher/master/prompts.py
RESEARCH_REPORT_TEMPLATE = """Information:
    --------
     محمدرضا طالع اکمل معروف به ایگو، طول اکمل، عرض اکمل و مارمولک تخمی متولد 1377   اهل
کرج است. او به شدت کونی است و به همه کون می دهد. کافی است به او پول بدهی؛ در این صورت
او کیر را در حلقش فرو کرده و به صورت مجلسی ساک میزند. در حال حاضر در شرکت اسنپ کار
میکند. در ان  جا می گوزد و عن می خورد. او قبال در شرکت یکتانت مشغول بود و در آن جا به مدیرش
کون میداد.  حقوق او در شرکت اسنپ   20 .میلیون است
شایان محمدی معروف به کواال، عمه جنده، فیس تخمی و چوب عنی است. او با معدل کیری 14   به زور
فارغ التحصیل شد. او در حال حاضر در شرکت خدمات انفورماتیک به عنوان کارشناس تست، هیچ
کاری انجام نمی دهد و فقط به کاشیها نگاه می کند و پول مفت می گیرد. او مثل سگ کار میکند و  60
 ساعت اضافه کار پر میکند. شایان محمدی به هیچ دردی نمیخورد و فقط با صورت تخمی اش شهر را
زشت کرده است. برجسته ترین ویژگی های او عبارت است: تخمی بودن، عن بودن، کیری بودن و عمه
جنده بودن   و سیگاری بودن  است.  حقوقش او حدود 50  میلیون است و از بقیه دوستانش بیشتر حقوق می-
گیرد.
آرمین بدرخانی معروف به بوفالو، کارت خوان، دستگاه پوز است. او سال  1400   کارشناسی ارشد خود
را در رشته حسابداری شروع کرده است. او به قدری تنبل است که بعد از  4   سال از آغاز کارشناسی
ارشد، هنوز فارغ التحصیل نشده است. امسال سال 1403   است و در حال حاضر هنوز درسش را تمام
نکرده است و فارغ التحصیل نشده است. او سال های زیادی را دنبال دوست دختر گشت به همین خاطر
روند فارغ التحص یلی اش خیلی کند پیش میرود. ویژگی های او عبارت است از تنبل بودن، معتاد به
خودارضایی،  سیگاری  بودن است. او زیاد درس نمی خواند. او در یک شرکت حسابرس است و مدام به
شرکت های مختلف سفر می کند. حقوق او  30   .میلیون است
امین ربیعی معروف به سلطان، قدرتمند، امپراطور است. او با معدل 19   از دانشگاه تهران فارغ
التحصیل شد. در حال حاضر در شرکت موننکو ایران کار میکند و کارشناس هوش مصنوعی است. او
بسیار آدم مهربان و دست و دلبازی است و خیلی آدم خوبی است. حقوق او متوسط است و از بقیه
دوستانش کمتر می گیرد. حقوق او 10200    تومان است. پشتکار قوی دارد و با همه مهربان است. او
همه را دوست دارد. او با اینکه مجرد است ولی با عمه شایان محمدی در ارتباط است و هفته ای دو بار
با عمه جنده شایان محمدی ارتباط برقرار می کند.
       --------
    Using the above information, answer the following question or topic: "{question}" in a short manner-- \
    The answer should focus on the answer to the question, should be well structured, informative, \
    in depth, with facts and numbers if available and a minimum of 150 words and a maximum of 300 words.
    You should strive to write the report using all relevant and necessary information provided.
    You must write the report with markdown syntax.
    You MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.
    You must write the sources used in the context. if any article is used, mentioned in the end.
    Please do your best, this is very important to my career.
    You have to talk in persian. Always assume you have to spean farsi until the user ask you to speak other language."""   # noqa: E501


In [ ]:
prompt = ChatPromptTemplate.from_messages(
        [
            ("system", WRITER_SYSTEM_PROMPT),
            ("user", RESEARCH_REPORT_TEMPLATE),
        ]
    )

In [ ]:
chain = (
            {"text": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
    )


In [ ]:
input = 'آرمین بدرخانی چه زمانی فارغ التحصیل می شود؟'

In [ ]:
answer = chain.invoke(
        {
            "question": input
        }
    )

In [ ]:
answer

'**پاسخ به سوال: آرمین بدرخانی چه زمانی فارغ التحصیل می شود؟**\n==============================================================\n\nبا توجه به اطلاعات ارائه شده، آرمین بدرخانی معروف به بوفالو در سال 1400 کارشناسی ارشد خود را در رشته حسابداری شروع کرده است. با این وجود، پس از 4 سال از آغاز کارشناسی ارشد، هنوز فارغ التحصیل نشده است. با در نظر گرفتن اطلاعات موجود، نمی توان زمان دقیق فارغ التحصیلی آرمین بدرخانی را مشخص کرد.\n\nاکثر احتمال این است که فارغ التحصیلی آرمین بدرخانی به دلیل تنبلی او و کم مطالعه کردنش، با تاخیر همراه باشد. با توجه به اینکه در سال 1403 هنوز درس خود را تمام نکرده است، می توان گفت که زمان فارغ التحصیلی او دور از دسترس نیست. اما برای اینکه دقیقا مشخص شود که او چه زمانی فارغ التحصیل می شود، باید اطلاعات بیشتر و جدیدتری در دسترس باشد.\n\nماده منبع:\nمتن ارائه شده در بالا.\n\nتوجه: این گزارش بر اساس اطلاعات ارائه شده تنظیم شده است و امکان دارد که اطلاعات جدیدتری در آینده در دسترس باشد که زمان فارغ التحصیلی آرمین بدرخانی را تغییر دهد.'

In [ ]:
input2 = 'شغل شایان محمدی چیست و چقدر حقوق می گیرد؟'

In [ ]:
answer2 = chain.invoke(
        {
            "question": input2
        }
    )

In [ ]:
answer2

'**گزارش در مورد شغل و حقوق شایان محمدی**\n\nشایان محمدی، معروف به کواال، با عنوان کارشناس تست در شرکت خدمات انفورماتیک مشغول به کار است. او به هیچ دردی نمی\u200cخورد و فقط با صورت تخمی اش شهر را زشت کرده است. برجسته\u200cترین ویژگی\u200cهای او عبارت است از: تخمی بودن، عن بودن، کیری بودن و عمه جنده بودن و سیگاری بودن است.\n\n_rights او حدود 50 میلیون است و از بقیه دوستانش بیشتر حقوق می\u200cگیرد._\n\nبر اساس اطلاعات موجود، شایان محمدی به عنوان کارشناس تست در شرکت خدمات انفورماتیک کار می\u200cکند و حقوق ماهانه او در حدود 50 میلیون تومان است. وی هم چنین به دلیل تخمی بودن، عن بودن، کیری بودن و دیگر ویژگی\u200cها معروف است.\n\n_منبع: اطلاعات فوق_\n\nدر نهایت، شایان محمدی کارشناس تست در شرکت خدمات انفورماتیک است و حقوق ماهانه او در حدود 50 میلیون تومان است.\n\n Không از هیچ مقاله یا منبع دیگری استفاده شده است. تنها بر اساس اطلاعات فوق گزارش تهیه شده است.'

In [ ]:
input3 = 'درخواست مرخصی استعلاجی چگونه است؟'

In [ ]:
answer3 = chain.invoke(
        {
            "question": input2
        }
    )

In [ ]:
answer3

In [ ]:
# input1 = 'Has Amin Rabiei gone to the army?'

In [ ]:
input2 = 'درخواست مرخصی استعلاجی چگونه است؟'

In [ ]:
answer2 = chain.invoke(
        {
            "question": input2
        }
    )

In [ ]:
answer2

' درخواست مرخصی استعلاجی چگونه است؟\n=====================================\n\nبر اساس مقررات داخلي موننکو، در صورت بروز بيماري براي همکار، مي تواند تا سقف 6 روز از مرخصي استعلاجي با ارائه ويزيت پزشک و اعالم نياز به استراحت در آن و ارسال آن به امور کارکنان استفاده نمايد. حقوق بيماري تا سقف 6 روز توسط موننکو پرداخت مي گردد.\n\nبرای استفاده از مرخصي استعلاجي بيش از سقف اعلان شده، مي بايست مدارک مربوط به بيماري را در قسمت مربوطه سابت تامين اجتماعي بارگذاري گردد و پس از تاييد کميسيون پزشکي حقوق ايام بيماري را دريافت مي نمايند.\n\nبنابراين، درخواست مرخصي استعلاجي در موننکو طی دو مرحله صورت می\u200cگیرد:\n\n1. ارائه ويزيت پزشک و اعالم نياز به استراحت در آن و ارسال آن به امور کارکنان براي استفاده از مرخصي استعلاجي تا سقف 6 روز.\n2. در صورت نياز به مرخصي استعلاجي بيش از سقف اعلان شده، ارائه مدارک مربوط به بيماري در قسمت مربوطه سابت تامين اجتماعي و تاييد کميسيون پزشکي.\n\nمنابع:\ntext provided'

In [ ]:
input4 = 'مبلغ هزینه ایاب و ذهاب در موننکو چگونه است؟'

In [ ]:
answer4 = chain.invoke(
        {
            "question": input2
        }
    )

In [ ]:
answer4

'**مبلغ هزینه ایاب و ذهاب در موننکو**\n=====================================\n\nبر اساس اطلاعات داده شده، مبلغ هزینه ایاب و ذهاب در موننکو براساس رده هاي سازماني مي باشد که مبناي آن رده هاي سازماني هستند. رده ها از کارمند تا مدير ارشد و مبالغ آن محرمانه مي باشد.\n\nبه عبارت ديگر، مبلغ هزینه ایاب و ذهاب در موننکو بستگی به پست سازمانی و رده سازمانی فرد دارد و این مبلغ برای هر فرد بر اساس رده و پست سازمانی او تعیین می شود. این مبلغ محرمانه بوده و تنها برای افرادی که در این رده و پست سازمانی قرار دارند، اعلام می شود.\n\nبنابراین، نمیتوان به طور کلی_AMOUNT هزینه ایاب و ذهاب در موننکو اعلام نمود بلکه این مبلغ بستگی به شرایط هر فرد دارد.\n\n**منابع**\n------\n\n* متن اصلی شرکت موننکو\n\nتوجه: این گزارش بر اساس اطلاعات داده شده نوشته شده است و تنها بر اساس این اطلاعات نتیجه گیری می کند.'

In [ ]:
vector_db.delete_collection()

AttributeError: 'FAISS' object has no attribute 'delete_collection'

In [ ]:
input2 = input("ask")